#**Importing**

In [ ]:
!pip install pyarabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 3.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, GlobalMaxPooling1D, LSTM, Embedding, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import tensorflow as tf 
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from pyarabic import araby
from pyarabic.araby import tokenize, is_arabicrange, strip_tashkeel
import time
from sklearn.metrics import f1_score, precision_score, recall_score

#**Data Preprocessing and Generation**

In [ ]:
# sentences = [
#     'أهلاً وسهلاً بكم في موقعنا.',
#     'نحن نقدر اهتمامكم بخدماتنا.',
#     'هل تحتاجون إلى مساعدة؟',
#     'نحن دائماً في خدمتكم.',
#     'شكراً لكم على زيارتكم لنا.',
#     'نحن نسعى جاهدين لتقديم أفضل الخدمات.',
#     'أهمية البيئة تزداد يوماً بعد يوم.',
#     'علينا جميعاً المحافظة على الأرض.',
#     'الصحة هي الثروة الحقيقية في الحياة.',
#     'ننصح بتناول الأطعمة الصحية وممارسة الرياضة.',
#     'الحياة قصيرة، لذلك يجب علينا الاستمتاع بها.',
#     'يجب الاهتمام بالصحة النفسية.',
#     'التقنية تتطور بسرعة.',
#     'يجب مواكبة التطور التكنولوجي.',
#     'العمل الجماعي يؤدي إلى النجاح.',
#     'يجب تحقيق التوازن بين العمل والحياة الشخصية.',
#     'التعلم مستمر في الحياة.',
#     'الوقت هو الأغلى في الحياة.',
#     'الحياة بلا أهداف ليست مجردة.',
#     'التغيير يجب أن يأتي من الداخل.',
#     'التعليم هو مفتاح النجاح في الحياة.',
#     'الاستماع هو مفتاح الاتصال الجيد.',
#     'الصداقة هي علاقة قيمة جداً في الحياة.',
#     'الحب والعطف يجعلان الحياة أكثر جمالاً.',
#     'التسامح يساعد على بناء علاقات جيدة.',
#     'العمل الجاد هو مفتاح النجاح.',
#     'يجب تحقيق التوازن بين العمل والحياة الشخصية.',
#     'العائلة هي المكان الذي نشعر بالانتماء إليه.',
#     'الحرية والديمقراطية هما أساس الحكم الرشيد.',
#     'الاهتمام بالتفاصيل يجعل الحياة أفضل.',
#     'الصلاة والتأمل يساعدان على الاسترخاء والتركيز.',
#     'الموسيقى هي لغة عالمية.',
#     'يجب علينا الحفظ على البيئة والحياة البرية.',
#     'التحدث بلغات متعددةيساعد على التواصل مع العالم.',
#     'الأدب والثقافة يغذيان الروح والعقل.',
#     'الابتسامة هي لغة عالمية.',
#     'التعاون يساعد على تحقيق الأهداف.',
#     'التفاؤل هو المفتاح للنجاح.',
#     'التخطيط الجيد هو أساس النجاح.',
#     'الصبر هو مفتاح الصمود في الحياة.',
#     'الصدق هو أساس الثقة والعلاقات الجيدة.',
#     'العمل الجماعي يساعد على تحقيق الأهداف.',
#     'الاستثمار في التعليم هو أساس تطور المجتمع.',
#     'الإيمان هو قوة الروح.',
#     'التغيير يجب أن يأتي من الداخل.',
#     'التسامح يساعد على بناء علاقات جيدة.',
#     'الإبداع والابتكار هما مفتاح التطور والتقدم.',
#     'يجب تقدير الوقت واستخدامه بشكل جيد.',
#     'العقل السليم في الجسم السليم هو الأساس للحياة الصحية.',
#     'التعليم هو أساس بناء المستقبل.',
#     'الاستماع الجيد يساعد على فهم الآخرين.',
#     'الجرأة هي المفتاح لتحقيق الأهداف الكبيرة.',
#     'الحفاظ على الأمان هو مسؤولية الجميع.',
#     'الإنجازات الكبيرة تأتي من خلال العمل الجاد.',
#     'علينا جميعاً أن نكون مسؤولين في حياتنا.',
#     'التركيز والانتباه يساعدانعلى تحقيق النجاح في الحياة.',
#     'التواضع هو مفتاح النجاح في الحياة.',
#     'يجب الاستماع للنصائح الحكيمة من الآخرين.',
#     'التحدث بصراحة هو أساس العلاقات الجيدة.',
#     'العطاء والتفاني هما مفتاح الخير في الحياة.',
#     'العمل الشاق هو مفتاح النجاح في الحياة.',
#     'الإرادة القوية هي المفتاح لتحقيق الأهداف.',
#     'المثابرة والاجتهاد هما مفتاح النجاح في الحياة.',
#     'التفكير الإيجابي يساعد على تحقيق النجاح.',
#     'التحدث بلطف واحترام يساعد على بناء العلاقات الجيدة.'
#     'العمل الذي يحبه المرء هو العمل الأفضل.',
#     'الحياة بلا أحلام ليست مجردة.',
#     'التعلم من الأخطاء هو مفتاح التطور.',
#     'العمل الشاق هو مفتاح النجاح في الحياة.',
#     'الصبر هو مفتاح الصمود في الحياة.',
#     'الصراحة هي أساس العلاقات الجيدة.',
#     'الحب هو أساس الحياة.',
#     'التسامح يجعل الحياة أفضل.',
#     'الأمل هو الشمعة التي تضيء في ظلام الحياة.',
#     'التغيير يجب أن يأتي من الداخل.',
#     'كر والتعاون يساعدان على تحقيق الأهداف.',
#     'الأمانة والصدق هما أساس العلاقات الجيدة.',
#     'الرضا بالقضاء والقدر يجعل الحياة أسهل.',
#     'العمل الذي يحبه المرء هو العمل الأفضل.',
#     'الصحة هي الثروة الحقيقية في الحياة.',
#     'التواضع هو مفتاح النجاح في الحياة.',
#     'الاحترام هو أساس العلاقات الجيدة.',
#     'الطموح هو المفتاح لتحقيق النجاح.',
#     'العمل الشاق يؤدي إلى النجاح.',
#     'الثقة بالنفس هي المفتاح لتحقيق الأهداف.',
#     'الحياة قصيرة، لذلك يجب الاستمتاع'
#     ,
#     'الأمانة والصدق هما أساس العلاقات الجيدة.',
#     'الرضا بالقضاء والقدر يجعل الحياة أسهل.',
#     'العمل الذي يحبه المرء هو العمل الأفضل.',
#     'الصحة هي الثروة الحقيقية في الحياة.',
#     'التواضع هو مفتاح النجاح في الحياة.',
#     'الاحترام هو أساس العلاقات الجيدة.',
#     'الطموح هو المفتاح لتحقيق النجاح.',
#     'العمل الشاق يؤدي إلى النجاح.',
#     'الثقة بالنفس هي المفتاح لتحقيق الأهداف.',
#     'الحياة قصيرة، لذلك يجب الاستمتاع بها.',
#     'التفاؤل هو المفتاح لتحقيق النجاح في الحياة.',
#     'التوازن في الحياة يساعد على السعادة والراحة.',
#     'التحدث بلباقة يساعد على بناء العلاقات الجيدة.',
#     'العمل الجماعي يساعد على تحقيق',' النجاح في الأعمال.',
#     'التفكير الإيجابي يساعد على تحقيق الإنجازات الكبيرة.',
#     'الصداقة',' الحقيقية تدوم طويلاً.',
#     'التحلي بالصبر والإصرار يساعد على تحقيق الأهداف.',
#     'العمل الذي يقوم به المرء بإخلاص يؤدي إلى النجاح.',
#     'الثقة بالآخرين هي أساس العلاقات الجيدة.',
#     'التعاون والتضامن يساعدان على تحقيق الأهداف الكبيرة.',
#     'التغيير هو جزء لا يتجزأ من الحياة.',
#     'العمل الجاد يؤدي إلى النجاح في الحياة.',
#     'التواضع والتقدير يساعدان على بناء العلاقات الجيدة.',
#     'العفوية والصراحة تجعل الحياة أكثر جمالاً.',
#     'الاستماع الجيد يساعد على فهم الآخرين بشكل أفضل.',
#     'التحدث بلباقة واحترام يساعد على بناء العلاقات الجيدة.',
#     'التخطيط الجيد يساعد على تحقيق الأهداف بشكل أسرع.',
#     'الحب والعطف يجعلان الحياة أكثر جمالاً وسعادة.',
#     'الابتسامة هي لغة عالمية يفهمها الجميع.',
#     'التعلم والتطوير المستمر يساعدان على تحقيق النجاح في الحياة.',
#     'العمل بجدية واجتهاد يساعد على تحقيق الأهداف.',
#     'التفكير الإيجابي يساعد على تخطي الصعاب والتحديات.',
#     'الاستعداد والتخطيط الجيد يجعلان الحياة أسهل وأكثر نجاحاً.',
#     'التعلم من الأخطاء يساعد على التحسين والتطور.',
#     'الاهتمام بالتفاصيل يساعد على تحقيق النجاح في الحياة.',
#     'التغلب على المخاوف والتحديات يساعد ','على تحقيق النجاح والنمو الشخصي.',
#     'الاحتفاء بالتنوع واحترام الاختلافات يجعل العالم أكثر تفاعلاً وتعاوناً.',
#     'التركيز على الحلول بدلاً من التركيز على ','المشاكل يساعد على تحقيق النجاح.',
#     'التعامل بحكمة مع الوقت يساعد على تحقيق',' المزيد من الإنجازات في الحياة.',
#     'الاستمتاع باللحظة الحالية يجعل ','الحياة أكثر سعادة وإيجابية.',
#     'التفاني والإخلاص في العمل يساعد',' على تحقيق النجاح المستدام.',
#     'العمل على تطوير الذات وتحسين الأداء يساعد',' على تحقيق النجاح في الحياة.',
#     'الاستماع للآراء والأفكار المختلفة يساعد',' على توسيع المدارك والتعلم.',
#     'التفكير المنطقي والاستنتاج الصحيح يساعد','ان على تحقيق النجاح في الحياة.',
#     'التحلي بالصدق والشفافية يساعد',' على بناء الثقة والعلاقات الجيدة.',
#     'التعلم من الخبرات السابقة يساعد ','على تحسين الأداء وتجنب الأخطاء.',
#     'التركيز على الأهداف الكبيرة والطموحة يساعد على تحقيق النجاح في الحياة.',

#     'الصبر والصمود في مواجهة الصعاب ','يساعدان على تحقيق النجاح.',
#     'التعامل بلطف وحسن تصرف يساعد',' على بناء العلاقات الجيدة.',
#     'التحلي بالشجاعة والاستعداد للمغامرة',' يساعدان على تحقيق الإنجازات الكبيرة.',
#     'التفكير الإبداعي والابتكار',' يساعدان على النجاح في الأعمال والحياة.',
#     'التحلي بالصبر والتفاؤل يساعد على تجاوز الصعاب وتحقيق النجاح.',
#     'الاستمتاع بالأشياء البسيطة في الحياة يجعل الحياة أكثر جمالاً وسعادة.',
#     'الاستماع للآخرين والتعلم منهم يساعد على توسيع المدارك',' وتحقيق النمو الشخصي.',
#     'التعاطف والتفهم يساعدان على',' بناء العلاقات الجيدة وتعزيز التعاون.',
#     'الاستمرار في التطوير والتعلم يساعد على ','التأقلم مع التغييرات وتحقيق النجاح.',
#     'التخطيط والتنظيم يساعدان على تحقيق ','الأهداف بشكل أكثر فعالية ونجاحًا.',
#     'التحلي بالكفاءة والمهارة يساعد على',' تحقيق النجاح في الأعمال والحياة.',
#     'التركيز على الحلول يساعد على',' تجاوز الصعاب وتحقيق النجاح في الحياة.',
#     'التحلي بالصداقة والإخلاص يساعد على بناء',' العلاقات الجيدة وتعزيز التعاون.',
#     'الاستمتاع بالعمل والحياة',' يجعل الحياة أكثر سعادة وإيجابية.',
#     'الاهتمام بالتغذية الصحية والنشاط البدني',' يساعد على الحفاظ على الصحة الجيدة والسعادة.',
#     'التفكير بشكل إيجابي والتركيز على الأمور الجيدة',' يساعد على تحقيق النجاح والسعادة.',
#     'التحلي بالتسامح والعفوية يساعد على تحسين ','العلاقات وتعزيز التعاون.',
#     'التعلم من الأخطاء والتحسين المستمر يساعد','على تحقيق النجاح في الحياة.',
#     'التحلي بالصبر والاستماع للآخرين يساعد على بناء',' العلاقات الجيدة وتعزيز التعاون.',
#     'الاستمرار في المحاولة وعدم ','الاستسلام يساعد على تحقيق النجاح في الحياة.',
#     'التحلي بالنضج والتفكير الواعي يساعد على ','اتخاذ القرارات الصحيحة وتحقيق النجاح.',
#     'الاستمتاع بالطبيعة والبيئة يجعل',' الحياة أكثر جمالاً وسعادة.',
#     'التحلي بالإيمان والتفاؤل يساعد على تجاوز',' الصعاب وتحقيق النجاح في الحياة.',
#     'التعلم وتطوير مهارات جديدة يساعد على تحقيق',' النجاح في الحياة المهنية والشخصية.',
#     'التعامل بالصدق والنزاهة يساعد على بناء الثقة',' والاحترام في العلاقات الشخصية والمهنية.',
#     'التحلي بالانضباط والتركيز يساعد على تحقيق',' الأهداف بشكل أكثر فعالية ونجاحًا.',
#     'التحلي بالإلهام والإثارة يساعد على تحفيز',' الذات وتحقيق الإنجازات الكبيرة.',
#     'التحلي بالتواضع والتقدير يساعد على بناء ','العلاقات الجيدة وتعزيز التعاون.',
#     'التعامل بالاحترام والتقدير يساعد على بناء',' العلاقات الجيدة وتعزيز التعاون.',
#     'الاستماع للمشاعروالآراء الأخرى بصدر رحب يساعد',' على تحسين العلاقات وتعزيز التفاهم.',
#     'التحلي بالصدق والشفافية في التعامل يساعد على بناء',' الثقة وتعزيز العلاقات الجيدة.',
#     'التحلي بالمرونة والتكيف مع التغيير يساعد على ','تحقيق النجاح في الحياة والأعمال.',
#     'الاستمرار في التطوير والتحسين يساعد على النمو',' الشخصي وتحقيق النجاح في الحياة.',
#     'الاستماع لنصائح الآخرين وتقبلها بصدر رحب يساعد',' على تحقيق النجاح في الحياة والأعمال.',
#     'التحلي بالمسؤولية والالتزام يساعد على تحقيق النجاح',' والاحترام في الحياة المهنية والشخصية.',
#     'التعامل بالحكمة والتفكير الواعي يساعد',' على اتخاذ القرارات الصحيحة وتحقيق النجاح.',
#     'التحلي بالصبر والتحمل في مواجهة الصعاب',' يساعد على تحقيق النجاح في الحياة.',
#     'التعامل بالإيجابية والتفاؤل يساعد على تحقيق',' النجاح في الحياة وتجاوز الصعاب.',
#     'التحلي بالصداقة والتعاون يساعد',' على بناء العلاقات الجيدة وتحق',
#     'يق النجاح في الحياة',' الشخصية والمهنية.',
#     'التحلي بالتواضع وعدم الغرور يساعد على بناء ','العلاقات الجيدة والتعاون مع الآخرين.',
#     'التحلي بالشجاعة والإصرار يساعد على تحقيق',' الأهداف الصعبة والنجاح في الحياة.',
#     'التفكير بشكل إيجابي وتحويل الصعاب إلى فرص ','يساعد على تحقيق النجاح في الحياة.',
#     'التحلي بالمرونة والتكيف مع التغييرات يساعد على ','التأقلم مع التحديات وتحقيق النجاح.',
#     'الاستمتاع بالوقت الحاضر وعدم القلق من المستقبل',' يجعل الحياة أكثر سعادة وإيجابية.',
#     'التحلي بالصبر والاستمرارية يساعد على تحقيق',' النجاح في الحياة وتجاوز الصعاب.',
#     'التواصل بفعالية وفهم الآخرين يساعد على بناء',' العلاقات الجيدة وتعزيز التعاون.',
#     'التحلي بالانضباط والتركيز يساعد على تحقيق ','الأهداف بشكل أكثر فعالية ونجاحًا.',
#     'التعلم من الأخطاء والتحسين المستمر يساعد',' على تحقيق النجاح في الحياة.',
#     'الاستمتاع بالعمل والاهتمام بالتفاصيل يجعل',' الحياة أكثر جمالاً وإيجابية.',
#     'التحلي بالصدق والنزاهة يساعدعلى بناء الثقة والاحترا','م في العلاقات الشخصية والمهنية.',
#     'التحلي بالاستقلالية والتفكير خارج الصندوق يساعد',' على تحقيق الإنجازات الكبيرة في الحياة.',
#     'التحلي بالتعاطف والترحيب بالآخرين يساعد على بناء ','العلاقات الجيدة والتعاون معهم.',
#     'التحلي بالتفاؤل والإيمان بالقدرات الشخصية يساعد',' على تحقيق النجاح في الحياة.',
#     'التحلي بالصبر والتفاني في العمل يساعد على تحقيق النجاح ','والتميز في المجال الذي تعمل فيه.',
#     'التحلي بالكفاءة والإتقان في العمل يساعد على تحقيق النجاح والاحترام',' في الحياة المهنية والشخصية.',
#     'التركيز على الحلول والإيجابية يساعد ','على تجاوز الصعاب وتحقيق النجاح في الحياة.',
#     'التحلي بالنظام والتنظيم يساعد على',' تحقيق الأهداف بشكل أكثر فعالية ونجاحًا.',
#     'التعلم من النجاح وتوظيف الخبرات في المستقبل يساعد ','على تحقيق المزيد من النجاح في الحياة.',
#     'التحلي بالصداقة والتعاون والتفاهم يساعد',' على بناء العلاقات الجيدة والاستمرار فيها.',
#     'التحلي بالاستماع للآخرين واحترام وجهات نظرهم يساعد على بناء العلاقات ','الجيدة وحل المشكلات بشكل أكثر فعالية.',
#     'التحلي بالتفكير الإيجابي والتفاؤل يساعد على تحقيق',' النجاح وتجاوز الصعوبات في الحياة.',
#     'التحلي بالانفتاح والتعلم المستمر يساعد على تحقيق',' النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتواضع',' والإخلاص يساعد على بناء الثقة',' والاحترام في العلاقات الشخصية والمهنية.',
#     'التحلي بالتفاني والتركيز على الأهداف يساعد على تحقيق ','النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالمبادرة والاستعدام الخمول',' يساعد على تحقيق الأهداف والنجاح في الحياة.',
#     'التحلي بالاستماع الفعال والتعرف على احتياجات الآخرين يساعد على بناء العلاقات الجيدة وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتنظيم والتخطيط ','يساعد على تحقيق الأهداف بشكل أكثر فعالية ونجاحًا.',
#     'التحلي بالتفهم والتسامح يساعد ','على بناء العلاقات الجيدة والتعاون مع الآخرين.',
#     'التحلي بالابتكار والإبداع يساعد على تحقيق النجاح وتحقيق',' الإنجازات الكبيرة في الحياة المهنية والشخصية.',
#     'التحلي بالحكمة والتفكير الاستراتيجي يساعد على اتخاذ القرارات الصائبة وتحقيق النجاح في الحياة.',
#     'التحلي بالاستمرارية وعدم الاستسلام يساعد على تحقيق ','النجاح في الحياة، حتى في الظروف الصعبة.',
#     'الاستماع إلى الآخرين وتقدير آرائهم يساعد على بناء العلاقات الجيدة وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتفكير النقدي والتحليلي يساع',

#     'د على اتخاذ القرارات الصائبة وتحقيق ','النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالصبر والهدوء في التعامل مع الصعاب يساعد',' على تجاوزها وتحقيق النجاح في الحياة.',
#     'التحلي بالتعاطف ','والتفهم يساعد على بناء العلاقات الجيدة',' والتعاون مع الآخرين في الحياة المهنية والشخصية.',
#     'التحلي بالصدق والامتنان يساعد على بناء الثقة والاحترام في العلاقات الشخصية والمهنية.',
#     'التحلي بالتواضع والاعتراف بالأخطاء',' يساعد على التطور والتحسين في الحياة المهنية والشخصية.',
#    'التحلي بالإيمان بالذات والثقة في القدرات ','الشخصية يساعد على تحقيق النجاح وتجاوز الصعوبات في الحياة.',
#     'التحلي بالتنظيم والتخطيط يساعد على تحقيق',' الأهداف بشكل أكثر فعالية ونجاحًا',' في الحياة المهنية والشخصية.',
#     'التحلي بالابتكار والإبداع يساعد على تحقيق',' النجاح وتحقيق الإنجازات الكبيرة في الحياة المهنية والشخصية.',
#     'التحلي بالحكمة والتفكير الاستراتيجي يساعد على اتخاذ',' القرارات الصائبة وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحليبالتفاعل الإيجابي والبناء مع الآخرين يساعد على بناء',' العلاقات الجيدة وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتعلم واكتساب المعرفة المستمر',' يساعد على تحقيق النجاح في الحياة والتطور المهني والشخصي.',
#     'التحلي بالتحليل والتفكير الناقد ','يساعد على اتخاذ القرارات الصائبة وتحقيق النجاح في الحياة.',
#     'التحلي بالتواضع والتعلم من الآخرين يساعد على التطور والتحسين في ','الحياة المهنية والشخصية.',
#     'التحلي بالتفاني والاجتهاد يساعد على تحقيق',' النجاح في الحياة المهنية والشخصية وتحقيق الأهداف المنشودة.',
#     'التحلي بالانفتاح والاستماع',' لآراء الآخرين يساعد على بناء',' العلاقات الجيدة والتعاون معهم في الحياة المهنية والشخصية.',
#     'التحلي بالتعاون والعمل الجماعي يساعد على تحقيق ','الأهداف بشكل أكثر فعالية ونجاحاً في الحياة المهنية والشخصية.',
#     'التحلي بالصبر والإصرار يساعد',' على تجاوز الصعاب وتحقيق النجاح',' في الحياة المهنية والشخصية.',
#     'التحلي بالابتكار والتفكير خارج ','الصندوق يساعد على الوصول إلى حلول',' جديدة وإيجاد فرص جديدة في الحياة المهنية والشخصية.',
#     'التحلي بالتأني وعدم الاستعجال في اتخاذ القرارات يساعد على اتخاذ القرارات الصائبة',' وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتركيز والانتباه ','يساعد على تحقيق الأهداف بشكل أكثر',' فعالية ونجاحاً في الحياة المهنية والشخصية.',
#     'التحلي بالصدق والشفافية في التعامل مع الآخرين يساعد على',' بناء الثقة والاحترام في العلاقات الشخصية والمهنية.',
#     'التحلي بالاستمتاع بالعمل والحياة يساعد على الشعور بالسعادة والرضا في الحياة المهنية والشخصية، ','وهذا يؤثر إيجاباً على الأداء والإنجازات.',
#     'التحلي بالإيجابية والتفاؤل يساعد على تحقيق النجاح وتجاوز الصعوبات في الحياة، ويؤثر',' إيجاباً على العلاقات الشخصية والمهنية.',
#     'التحلي بالاستمرارية وعدم الاستسلام للفشل يساعد على تجاوز الصعوبات وتحقيق ','النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتعامل بحكمة مع','النجاح وعدم الاستعجال في ','اتخاذ القرارات المهمة يساعد على تحقيق النجاح بطريقة',' مستدامة ومنظمة في الحياة المهنية والشخصية.',
#     'التحلي بالاهتمام بالصحة النفسية والجسدية يساعد على تحقيق ','النجاح في الحياة المهنية والشخصية،',' ويساعد على التحمل والتحمل في مواجهة التحديات.',
#     'التحلي بالتواضع وعدم الغرور يساعد على بناء العلاقات الجيدة',' وتحقيق النجاح في الحياة المهنية والشخصية،',' ويساعد على الاحترام والتقدير من الآخرين.',
#     'التحلي بالتفكير الإيجابي والتركي',
#   'ة يساعد على تحقيق النجاح وتحويل الصعاب',' إلى فرص في الحياة المهنية والشخصية.',
#     'التحلي بالصبر والتركيز على الهدف يساعد على تحقيق ','النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتفاؤل والثقة في المستقبل يساعد',' على تجاوز الصعوبات وتحقيق ','النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتفكير',' الإيجابي يساعد على تحسين','الحالة المزاجية وتحقيق ','النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتواضع يساعد',' على التعلم من الآخرين وتحقيق',' النجاح في الحياة المهنيةوالشخصية.',
#     'التحلي بالانضباط والتزام الوقت يساعد على تحقيق الأهداف والنجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتفكير الناقد يساعد على تطوير الذات ','وتحسين الأداء في الحياة المهنية والشخصية.',
#     'التحلي بالتواضع والتعاون مع الآخرين يساهم في بناء فريق عمل ناجح في الحياة المهنية.',
#     'التحلي بالصدق والنزاهة يساعد على بناء الثقة',' والاحترام في العلاقات الشخصية والمهنية.',
#     'التحلي بالتفكير المنهجي والعلمي يساعد على ','اتخاذ القرارات الصائبة وتحقيق النجاح في الحياة ','المهنية والشخصية بشكل أكثر دقة وفعالية.',
#     'التحلي بالشجاعة والاستعداد للمخاطرة يساعد على اكتشاف ','فرص جديدة وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالاستراتيجية والتخطيط الجيد يساعد على تحقيق الأهداف بشكل أكثر فعالية ونجاحاً في الحياة المهنية والشخصية.',
#     'التحلي بالتفكير الشامل والنظر إلى الأمور من زوايا مختلفة يساعد على',' اتخاذ القرارات الصائبة وتحقيق النجاح في ','الحياة المهنية والشخصية.',
#     'التحلي بالتحفظ والتدريب المستمر يساعد على تحسين المهارات',' والأداء في الحياة المهنية والشخصية.',
#     'التحلي بالعمل الجاد ','والمثابرة يساعد على تحقيق ','النجاح في الحياة المهنية ','والشخصية وتحقيق الأهداف المرسومة.',
#     'التحلي بالتركيز على الحاجات والمتطلبات ','الأساسية يساعد على تحقيق النجاح في الحياة',' المهنية والشخصية وتحسين الجودة الحياتية.',
#     'التحلي بالتخصص والتعمق في المجال المهني',' يساعد على تحقيق النجاح والتميز في الحياة المهنية.',
#     'التحلي بالتفكير المبتكر والابتكاري يساعد على إيجاد حلول جديدة للمشكلات وتحقيق النجاح في ','الحياة المهنية والشخصية.',
#     'التحلي بالتعاطف والتفهم مع',' الآخرين يساعد على بناء العلاقات ','الإيجابية وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالاستماع الفعال والتواصل',' الجيد يساعد على بناء العلاقات ','الإيجابية وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتفاعل الإيجابي والبناء ','مع الآخرين يساعد على بناء العلاقات ','الإيجابية وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالمرونة والتأقليمية يساعد على التكيف مع التغييرات وتحقيق ','النجاح في الحياة المهنية والشخصية.',
#     'التحلي',' بالانفتاح والتعلم المستمر يساعد على','تحسين المهارات والأداء في الحياة المهنية والشخصية.',
#     'التحلي بالتقدير والاحترام للآخرين',' يساعد على بناء العلاقات ','الإيجابية وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتغلب على الخوف والشك ','يساعد على اتخاذ القرارات ','الصائبة وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالقدرة على إدارة الوقت ','والتحكم فيه يساعد على زيادة ','الإنتاجية وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالإلتزام بالأخلاقيات المهنية يساعد على بناء',' الثقة والاحترام في العلاقات المهنية وتحقيق النجاح.',
#     'التحلي بالإبداع والإبتكار يساعد على إيجاد حلول جديدة',' للمشكلات وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتركيز على الجوانب الإيجابية',' في الحياة يساعد على تحسين الحالة المزاجية وتحقيق الن',
#     'جاح في الحياة المهنية والشخصية.',
#     'التحلي بالتفكير الإبداعي يساعد على إيجاد حلول جديدة',' وإيجابية للمشكلات في الحياة المهنية والشخصية.',
#     'التحلي بالتفكير الناقد يساعد على تحليل الأمور بشكل دقيق واتخاذ ','القرارات الصائبة في الحياة المهنية والشخصية.',
#     'التحلي بالتعلم والتطوير المستمر يساعد على تحسين',' المهارات والكفاءات في الحياة المهنية والشخصية.',
#     'التحلي بالصبر والاستمرارية يساعد على تحقيق الأهداف',' في الحياة المهنية والشخصية بشكل دائم ومستمر.',
#     'التحلي بالتخطيط الجيد وتحديد الأولويات يساعد على تحقيق الأهداف',' بشكل أكثر فعالية في الحياة المهنية والشخصية.',
#     'التحلي بالصدق والشفافية يساعد على بناء',' الثقة والاحترام في العلاقات الشخصية والمهنية.',
#     'التحلي بالتواضع والتعاون يساعد',' على بناء فريق عمل ناجح في الحياة المهنية.',
#     'التحلي بالإبداع والابتكار يساعد على تطوير',' منتجات وخدمات مبتكرة في الحياة المهنية.',
#     'التحلي بالتركيز على الحلول بدلاً من المشاكل يساعد على تحقيق ','النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالاهتمام بالتفاصيل الصغيرة',' يساعد على تحقيق الدقة ','والاحترافية في الحياة المهنية والشخصية.',
#     'التحلي بالتواضع والترحيب بالانتقادات البناءة يساعد على تحسين الأداء',' والتطوير الذاتي في الحياة المهنية والشخصية.',
#     'التحلي بالجرأة والمبادرة يساعد على تحقيق الأهداف ','واكتشاف فرص جديدة في الحياة المهنية والشخصية.',
#     'التحلي بالتركيز على الحاجات والمتطلبات الأساسية للآخرين يساعد على بناء العلاقات الإيجابية ','وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتفكير الاستراتيجي والتخطيط المستقبلي يساعد على تحديد الأهداف وتحقيقها ','بشكل أكثر فعالية في الحياة المهنية والشخصية.',
#     'التحلي بالتركيز على الإيجابية',' وتجاهل السلبية يساعد على تحسين الحالة',' المزاجية وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتفكير المستقبلي ','والتطلع إلى الأفضل يساعد على رؤية ','الفرص وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتركيز على الحلول بدلاً من الشكاوى ','يساعد على تحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتواضع والاحترام للآخرين يساعد على بناء العلاقات ','الإيجابية وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتفكير الإيجابي وتحويل التحديات إلى فرص ','يساعد على تحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالصبر والتحلي',' بالصمود يساعد على تحمل ','الصعاب وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالاستماع الفعال يساعد على بناء العلاقات الإيجابية ','وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتعلم من الأخطاء يساعد على تطوير ','الذات وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتخطيط المنهجي وتحديد الخطوات ','اللازمة يساعد على تحقيق الأهداف بشكل أكثر فعالية',' في الحياة المهنية والشخصية.',
#     'التحلي بالتعامل الإيجابي مع الضغوط والتحديات يساعد على تحملها ','وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتعامل بحكمة وحسن تقدير مع الموارد المتاحة يساعد على ','استخدامها بشكل أكثر فعالية في الحياة المهنية والشخصية.',
#     'التحلي بالتكيف مع التغييرات ','والتطورات الحياة المهنية ','والشخصية يساعد على التأقلم وتحقيق النجاح.',
#     'التحلي بالتواصل الفعال يساعد على بناء العلاقات الإيجابية',' وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتفاني والتفرغ للعمل يساعد على تحقيق الأهداف والنجاح في الحياة المهنية.',
#     'التحلي بالتفكير المنطقي',' والتحليلي يساعد على فهم الأمور بشكل أكبر واتخاذ',' القرارات الصائبة في الحياة المهنية والشخصية.',
#     'التحلي بالتعاطف والتفهم يساعد على بناء',' العلاقات الإيجابية وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتحليل النفسي للذات يساعد على تحديد نقاط ','القوة والضعف وتطوير الذات في الحياة المهنية والشخصية.',
#     'التحلي بالتفكير الفعال والحلول الإبداعية يساعد',' على تطوير الأفكار والمشاريع في الحياة المهنية والشخصية.',
#     'التحلي بالتفاعل الإيجابي مع الآخرين يساعد على بناء الثقة',' والاحترام وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتركيزعلى الهدف ','والتخطيط الجيد يساعد على تحقيق ','الأهداف بشكل أكثر فعالية في الحياة المهنية والشخصية.',
#     'التحلي بالتعلم من الآخرين وتبادل الخبرات يساعد',' على تطوير المهارات والكفاءات في الحياة المهنية والشخصية.',
#     'التحلي بالتصميم الجيد والابتكار يساعد',' على تطوير منتجات وخدمات ','مبتكرة في الحياة المهنية.',
#     'التحلي بالتواصل الفعال والتعاون ','الجيد يساعد على بناء',' فرق عمل ناجحة في الحياة المهنية.',
#     'التحلي بالتركيز على الجودة والاحترافية يساعد على',' تحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتفكير الإستراتيجي والتخطيط المستقبلي يساعد على تحديد الأهداف والوصول ','إليها بشكل أكثر فعالية في الحياة المهنية والشخصية.',
#     'التحلي بالتركيز على العملاء وتلبية',' احتياجاتهم يساعد على تحقيق النجاح في الحياة المهنية.',
#     'التحلي بالتفكير المستقبلي والتطلع إلى الأفضل يساعد على رؤية ','الفرص وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتفكير الإيجابي والتفاؤل يساعد على تحسين الحالة ','المزاجية وتحقيق النجاحفي الحياة المهنية والشخصية.',
#     'التحلي بالتركيز على الحلول الإيجابية',' والابتكارية يساعد على تجاوز ','المشاكل وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتعاون والتواصل الجيد مع الزملاء والفريق يساعد على بناء',' بيئة عمل إيجابية وتحقيق النجاح في الحياة المهنية.',
#     'التحلي بالتركيز على تحقيق',' الأهداف وتحديد الأولويات يساعد على استثمار',' الوقت والجهد بشكل فعال في الحياة المهنية والشخصية.',
#     'التحلي بالتحليل النفسي والتطوير الذاتي يساعد على تحسينالذات وتطوير ','المهارات والكفاءات في الحياة المهنية والشخصية.',
#     'التحلي بالصبر والاستمرارية ','يساعد على تجاوز الصعوبات ','وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتفاعل الإيجابي مع التغييرات ','والتكيف معها يساعد على التطور والنمو في الحياة المهنية والشخصية.',
#     'التحلي بالتفكير النقدي والاهتمام بالتفاصيل يساعد على',' اتخاذ القرارات الصحيحة في الحياة المهنية والشخصية.',
#     'التحلي بالتواضع والاحترام والصدق يساعد على بناء علاقات إيجابية وتحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالاستماع الفعال والتعلم من الآخرين يساعد على تحسين ','المهارات الاجتماعية والتواصل في الحياة المهنية والشخصية.',
#     'التحلي بالتفكير الإبداعي والابتكاري يساعد على تطوير',' أفكار جديدة وحلول مبتكرة في',' الحياة المهنية والشخصية.',
#     'التحلي بالتخطيط الجيد والتنظيم يساعد على إدارة ','الوقت والموارد بشكل فعال في الحياة المهنية والشخصية.',
#     'التحلي بالتركيز على النتائج وتحقيق الأهداف',' يساعد على تحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتعلم المستمر وتطوير المهارات يساعد على ','الاستمرار في التحسين والتطور في الحياة المهنية والشخصية.',
#     'التحلي بالتفكير بشكل إيجابي وتحويل التحديات إلى فرص',' يساعد على تحقيق النجاح في الحياة المهنية والشخصية.',
#     'التحلي بالتركيز على الابتكار والتطوير يساعد على تطوير',' المنتجات والخدمات وتحقيق النجاح في الحياة المهنية.',
#     'التحلي بالتعاطف والاحترام والتفهم يساعد على بناء',



# ]

In [ ]:
# len(sentences)

In [ ]:
# zeros = [0.] * 550
# d = {"Text": sentences, "Language":zeros}
# ar_data = pd.DataFrame(d)
# ar_data

In [ ]:
df1 = pd.read_csv("/content/Language_det_train.csv")

In [ ]:
df1

,Text,Language
0,στη Γαλλία νωρίτερα ραντεβού χρησιμοποιήθηκε α...,Greek
1,e con ciò lei salì nella sua carrozza e senza ...,Italian
2,buna değmez.,Turkish
3,Viktiga skillnader är att i en wiki lagras sid...,Sweedish
4,تعرف على ما إذا كان شخص ما يقول نكتة رائعة يمك...,Arabic
...,...,...
9815,ഇംഗ്ലീഷ് വിക്കിപീഡിയയിൽ പലപ്പോഴും ഭൂരിപക്ഷം ആള...,Malayalam
9816,"Les algorithmes utilisés permettent, dans une ...",French
9817,"تم استخدام مصطلح ""التعلم الآلي"" لأول مرة في عا...",Arabic
9818,De hade under år 2000 försökt starta uppslagsv...,Sweedish


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9820 entries, 0 to 9819
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      9820 non-null   object
 1   Language  9820 non-null   object
dtypes: object(2)
memory usage: 153.6+ KB


In [ ]:
Lang = ['Arabic', 'English'] 

df2 = df1[df1["Language"].isin(Lang)]

In [ ]:
# df_concatenated = pd.concat([df2, ar_data])
# df2 = df_concatenated.sample(frac=1).reset_index(drop=True)

In [ ]:
# df2

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
def tokenize(text):
    return word_tokenize(text.lower())

def remove_stopwords_English(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word not in stop_words]

def remove_stopwords_Arabic(tokens):
    stop_words = set(stopwords.words('arabic'))
    return [word for word in tokens if word not in stop_words]

def remove_punctuation(tokens):
    punctuation = set(string.punctuation + '،؛؟')
    return [word for word in tokens if word not in punctuation]

def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in tokens]

def remove_numbers(tokens):
    return [re.sub(r'\d+', '', word) for word in tokens]

def preprocess(text,l_class):
    text = re.sub(r'http\S+', '', text)          # Remove URLs
    text = re.sub(r'@[^\s]+', '', text)          # Remove usernames
    text = re.sub(r'#(\w+)', '', text)           # Remove hashtags
    text = re.sub(r'[^\w\s]', '', text)          # Remove all non-alphanumeric characters
    tokens = re.findall(r'\b\w+\b', text)
    tokens = remove_numbers(tokens)
    if l_class == "English":
      tokens = remove_stopwords_English(tokens)
    elif l_class == "Arabic":
      tokens = remove_stopwords_Arabic(tokens)

    tokens = remove_punctuation(tokens)
    tokens = lemmatize(tokens)
    return tokens

In [ ]:
texts = df2["Text"]
lang_class = df2["Language"]
preprocessed_texts = []
for text, l_class in zip(texts,lang_class):
    preprocessed_text = preprocess(text,l_class)
    preprocessed_texts.append(' '.join(preprocessed_text[0:5]))

In [ ]:
df2["Text_Cleaned"] = preprocessed_texts

<ipython-input-866-acf30dac0cb7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Text_Cleaned"] = preprocessed_texts


In [ ]:
df2.head(10)

,Text,Language,Text_Cleaned
4,تعرف على ما إذا كان شخص ما يقول نكتة رائعة يمك...,Arabic,تعرف شخص يقول نكتة رائعة
8,you were saying .if somebody got a job of his ...,English,saying somebody got job dream
16,هذا سؤال جيد.,Arabic,سؤال جيد
17,[23] عزا فريق في مركز بالو ألتو للأبحاث هذا ال...,Arabic,عزا فريق مركز بالو
25,إنها جيدة بالنسبة لك ، سيكون أفضل بدونك وهذا م...,Arabic,إنها جيدة بالنسبة سيكون أفضل
29,Content in Wikipedia is subject to the laws (i...,English,Content Wikipedia subject law particular
31,[290] It was discontinued in February 2018.,English,It discontinued February
34,"تم تدوين سياسة ويكيبيديا الخاصة بـ ""وجهة نظر م...",Arabic,تم تدوين سياسة ويكيبيديا الخاصة
37,وتعتمد ويكيبيديا على برنامج ميدياويكي، وهو برن...,Arabic,وتعتمد ويكيبيديا برنامج ميدياويكي برنامج
51,"[353] In 2015, French researchers José Lages o...",English,In French researcher


In [ ]:
# Extract the text cleaned 
texts = df2["Text_Cleaned"]
# Extract target labels
df2["labels"] = df2["Language"]
# Mapping evert class name to a specific number
df2["labels"] = df2["labels"].replace(["Arabic"],[0.])
df2["labels"] = df2["labels"].replace(["English"],[1.])

<ipython-input-868-35f74e98107c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["labels"] = df2["Language"]
<ipython-input-868-35f74e98107c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["labels"] = df2["labels"].replace(["Arabic"],[0.])
<ipython-input-868-35f74e98107c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [ ]:
# Check the data frame data
df2

,Text,Language,Text_Cleaned,labels
4,تعرف على ما إذا كان شخص ما يقول نكتة رائعة يمك...,Arabic,تعرف شخص يقول نكتة رائعة,0.0
8,you were saying .if somebody got a job of his ...,English,saying somebody got job dream,1.0
16,هذا سؤال جيد.,Arabic,سؤال جيد,0.0
17,[23] عزا فريق في مركز بالو ألتو للأبحاث هذا ال...,Arabic,عزا فريق مركز بالو,0.0
25,إنها جيدة بالنسبة لك ، سيكون أفضل بدونك وهذا م...,Arabic,إنها جيدة بالنسبة سيكون أفضل,0.0
...,...,...,...,...
9804,The most obvious economic effect of Wikipedia ...,English,The obvious economic effect Wikipedia,1.0
9805,من دواعي سروري.,Arabic,دواعي سروري,0.0
9808,"Among Wikipedia editors, this is often phrased...",English,Among Wikipedia editor often phrased,1.0
9810,Multilinear subspace learning algorithms aim t...,English,Multilinear subspace learning algorithm aim,1.0


In [ ]:
# df2.loc[:,["Text","Text_Cleaned",'labels']].to_csv('filename.csv', index=False)

# **Model**

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(df2["Text_Cleaned"], df2['labels'], test_size=0.2, random_state=42, shuffle=True)

In [ ]:
# # tokenize the text
# tokenizer = Tokenizer(num_words=1000)
# tokenizer.fit_on_texts(X_train)

# X_train = tokenizer.texts_to_sequences(X_train)
# X_test = tokenizer.texts_to_sequences(X_test)

# # pad the sequences
# max_len = 100
# X_train = pad_sequences(X_train, maxlen=max_len)
# X_test = pad_sequences(X_test, maxlen=max_len)

# # Convert the labels to one-hot encoded vectors
# labels_train = np_utils.to_categorical(y_train)
# labels_test = np_utils.to_categorical(y_test)

In [ ]:
# tokenize the text
texts = df2['Text_Cleaned']
labels = df2['labels']
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
data = pad_sequences(sequences, maxlen=max_length)
# Convert the labels to one-hot encoded vectors
labels = np_utils.to_categorical(labels)

In [ ]:
# Define the model
model = Sequential()
model.add(Embedding(1000, 64, input_length=max_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_76"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_76 (Embedding)    (None, 5, 64)             64000     
                                                                 
 conv1d_74 (Conv1D)          (None, 1, 128)            41088     
                                                                 
 global_max_pooling1d_65 (Gl  (None, 128)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_148 (Dense)           (None, 10)                1290      
                                                                 
 dense_149 (Dense)           (None, 2)                 22        
                                                                 
Total params: 106,400
Trainable params: 106,400
Non-trainable params: 0
_______________________________________________

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

In [ ]:
  # Train the model
model.fit(data, labels, epochs=10, batch_size=32)

Epoch 1/10
58/58 [==============================] - 1s 3ms/step - loss: 0.6035 - accuracy: 0.7173
Epoch 2/10
58/58 [==============================] - 0s 3ms/step - loss: 0.2327 - accuracy: 0.9118
Epoch 3/10
58/58 [==============================] - 0s 3ms/step - loss: 0.0642 - accuracy: 0.9775
Epoch 4/10
58/58 [==============================] - 0s 3ms/step - loss: 0.0553 - accuracy: 0.9715
Epoch 5/10
58/58 [==============================] - 0s 3ms/step - loss: 0.0480 - accuracy: 0.9792
Epoch 6/10
58/58 [==============================] - 0s 3ms/step - loss: 0.0430 - accuracy: 0.9764
Epoch 7/10
58/58 [==============================] - 0s 3ms/step - loss: 0.0464 - accuracy: 0.9759
Epoch 8/10
58/58 [==============================] - 0s 3ms/step - loss: 0.0451 - accuracy: 0.9764
Epoch 9/10
58/58 [==============================] - 0s 3ms/step - loss: 0.0453 - accuracy: 0.9781
Epoch 10/10
58/58 [==============================] - 0s 3ms/step - loss: 0.0463 - accuracy: 0.9764


In [ ]:
# # evaluate the model on the testing set


# y_pred = model.predict(X_test)
# y_pred = np.round(y_pred).astype(int)
# f1 = f1_score(labels_test, y_pred,average = 'macro')
# precision = precision_score(labels_test, y_pred,average = 'macro')
# recall = recall_score(labels_test, y_pred,average = 'macro')

# print(f'F1 Score: {f1:.3f}')
# print(f'Precision: {precision:.3f}')
# print(f'Recall: {recall:.3f}')
# # [None, 'micro', 'macro', 'weighted', 'samples'].

15/15 [==============================] - 0s 6ms/step
F1 Score: 0.954
Precision: 0.953
Recall: 0.958


In [ ]:
test_texts =['hello', 'انا لا اشعر بالعطش']
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_data = pad_sequences(test_sequences, maxlen=max_length)
start_time = time.time()

predictions = model.predict(test_data)

end_time = time.time()

inference_time = end_time - start_time

print('Inference time:', inference_time, 'seconds')

# Print the predictions
for i in range(len(test_texts)):
    
    if np.argmax(predictions[i]) == 0:
        print(test_texts[i]," =================== ",'Arabic')
    elif np.argmax(predictions[i]) == 1:
        print(test_texts[i]," =================== ",'English')
    else:
      print("not supported")

1/1 [==============================] - 0s 81ms/step
Inference time: 0.14113306999206543 seconds
hello  ===================  English
انا لا اشعر بالعطش  ===================  Arabic


# **SAVING THE MODEL**

In [ ]:

export_path_keras = "/content/language_classification.h5"
print(export_path_keras)
model.save(export_path_keras)

/content/language_classification.h5


In [ ]:
# saving
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
max_length

5